In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta


In [2]:
# Random seed
random.seed(42)
np.random.seed(42)

# Medication specs
medication_specs = {
    "Fluoxetine 20mg": {"particle_mean": 80, "particle_std": 10, "pH_range": (5.5, 7.5)},
    "Sertraline 50mg": {"particle_mean": 90, "particle_std": 15, "pH_range": (6.0, 8.0)},
    "Alprazolam 0.5mg": {"particle_mean": 20, "particle_std": 5, "pH_range": (4.0, 5.5)},
    "Lorazepam 1mg": {"particle_mean": 25, "particle_std": 6, "pH_range": (4.0, 5.0)},
    "Clonazepam 0.5mg": {"particle_mean": 30, "particle_std": 5, "pH_range": (5.0, 6.5)},
    "Methylphenidate 10mg": {"particle_mean": 75, "particle_std": 12, "pH_range": (6.0, 7.5)},
    "Gabapentin 300mg": {"particle_mean": 100, "particle_std": 20, "pH_range": (5.0, 6.0)},
    "Topiramate 25mg": {"particle_mean": 110, "particle_std": 25, "pH_range": (5.0, 7.0)},
    "Fluoxetin 20mg": {"particle_mean": 80, "particle_std": 10, "pH_range": (5.5, 7.5)},
    "Sertralin 50mg": {"particle_mean": 90, "particle_std": 15, "pH_range": (6.0, 8.0)},
    "Alprazolam 0.50mg": {"particle_mean": 20, "particle_std": 5, "pH_range": (4.0, 5.5)},
    "Methylphenidate 10m": {"particle_mean": 75, "particle_std": 12, "pH_range": (6.0, 7.5)},
    "Topiramate 25mg": {"particle_mean": 110, "particle_std": 25, "pH_range": (5.0, 7.0)},
}

departments = ["QA", "QC Lab", "Production", "Packaging", "R&D"]
deviation_categories_by_department = {
    "QA": ["Human Error", "Process Error", "Quality Control"],
    "QC Lab": ["Equipment Failure", "Material Issue", "Quality Control"],
    "Production": ["Process Error", "Human Error", "Equipment Failure"],
    "Packaging": ["Human Error", "Process Error"],
    "R&D": ["Quality Control", "Material Issue"]
}

root_cause_by_category = {
    "Human Error": ["Training Gap", "Procedure Not Followed", "Operator Error"],
    "Equipment Failure": ["Equipment Malfunction"],
    "Material Issue": ["Material Contamination"],
    "Process Error": ["Procedure Not Followed", "Training Gap"],
    "Quality Control": ["Training Gap", "Operator Error"],
    "Humna Error": ["Training Gap", "Operator Error"]  # typo retained intentionally
}

severity_by_category = {
    "Human Error": ["Minor", "Moderate"],
    "Equipment Failure": ["Moderate", "Critical"],
    "Material Issue": ["Minor", "Moderate"],
    "Process Error": ["Moderate", "Critical"],
    "Quality Control": ["Minor", "Moderate", "Critical"],
    "Humna Error": ["Minor", "Moderate"]
}

investigators = ["J. Smith", "L. Zhang", "M. Patel", "D. Garcia", "A. Khan", "R. Evans", "S. Thomas"]
date_range = pd.date_range(datetime(2023, 1, 1), datetime(2024, 6, 30), freq='D')

def generate_dirty_data(num_records):
    data = []
    batch_to_product = {}
    medications = list(medication_specs.keys())

    for _ in range(num_records):
        open_date = random.choice(date_range)
        resolution_days = random.choice([random.randint(1, 10), random.randint(11, 100), 1000, np.nan])
        closed_date = open_date + timedelta(days=int(resolution_days)) if not pd.isna(resolution_days) else np.nan

        product_name = random.choice(medications)
        specs = medication_specs[product_name]

        # 1 batch = 1 product
        while True:
            batch_number = f"Batch_{random.randint(100, 999)}"
            if batch_number not in batch_to_product or batch_to_product[batch_number] == product_name:
                batch_to_product[batch_number] = product_name
                break

        department = random.choice(departments)
        deviation_category = random.choice(deviation_categories_by_department[department])
        root_cause = random.choice(root_cause_by_category[deviation_category])
        severity = random.choice(severity_by_category[deviation_category])

        # CAPA logic
        if severity == "Critical" or (deviation_category in ["Equipment Failure", "Process Error"] and root_cause in ["Training Gap", "Equipment Malfunction"]):
            capa_required = "Yes"
        else:
            capa_required = random.choices(["Yes", "No", np.nan], weights=[0.3, 0.6, 0.1])[0]

        capa_closed = random.choice(["Yes", "No"]) if capa_required == "Yes" else np.nan
        capa_number = f"CAPA-{random.randint(1000, 9999)}" if capa_required == "Yes" else np.nan

        if random.random() < 0.05:
            product_name = np.nan
        elif isinstance(product_name, str) and random.random() < 0.05:
            product_name = f" {product_name} "

        open_date_str = open_date.strftime('%m/%d/%Y') if random.random() < 0.5 else open_date.strftime('%d/%m/%Y')
        closed_date_str = closed_date.strftime('%m/%d/%Y') if isinstance(closed_date, datetime) else closed_date

        particle_size = np.random.normal(loc=specs["particle_mean"], scale=specs["particle_std"])
        pH = np.random.uniform(*specs["pH_range"])

        data.append([
            open_date_str, closed_date_str, batch_number, product_name, department,
            deviation_category, root_cause, severity,
            capa_required, capa_closed, capa_number, random.choice(investigators),
            particle_size, pH
        ])

    df = pd.DataFrame(data, columns=[
        "Open Date", "Closed Date", "Batch Number", "Product Name", "Department",
        "Deviation Category", "Root Cause", "Impact Severity",
        "CAPA Required", "CAPA Closed", "CAPA Number", "Investigator",
        "Particle Size (μm)", "pH Level"
    ])

    # Add duplicates
    df = pd.concat([df, df.sample(5, random_state=42)], ignore_index=True)

    # Shuffle
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    # Save
    df.to_excel("pharma_dirty_realistic_data.xlsx", index=False)
    return df

# Run this
df = generate_dirty_data(2500)
